In [37]:
#Importing libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from wordcloud import WordCloud,STOPWORDS
import sqlite3
import os
import requests

In [38]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('liSqdZAKoID2d-bIQH5M9A', 'tecIJkvcWwnQK9exC0fHNCnudtdYVA')

In [39]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password'}

In [40]:
with open('pw.txt', 'r') as file:
# Step 2: Read the contents of the file
        lines = file.readlines()

        # Iterate over each line
        for line in lines:
                # Split the line into username and password
                username, password = line.strip().split(':')

                # Store the username and password in the dictionary
                data['username'] = username
                data['password'] = password

In [41]:
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MySideProject/0.0.1'}

In [42]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

In [43]:
# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

In [44]:
# add authorization to our headers dictionary
headers['Authorization']=f'bearer {TOKEN}'

In [45]:
def df_from_response(res):
    # initialize temp list to store dictionaries
    data_list = []

    # loop through each post pulled from res and append to data_list
    for post in res.json()['data']['children']:
        data_list.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        })

    return data_list

Subreddits to explore

- politics
- worldnews
- news
- energy
- environment
- business
- worldevents

In [46]:
def extract_and_add_to_database(subreddit_name, conn):
    # Initialize dataframe and parameters for pulling data in loop
    data = pd.DataFrame()
    params = {'limit': 100}

    # Loop through 10 times (returning 1K posts)
    for i in range(10):
        # Make request to the specified subreddit
        res = requests.get(f"https://oauth.reddit.com/r/{subreddit_name}/new",
                           headers=headers,
                           params=params)

        # Get list of dictionaries from response
        data_list = df_from_response(res)

        # Convert list of dictionaries to DataFrame and append to data
        data = pd.concat([data, pd.DataFrame(data_list)], ignore_index=True)

        # Take the final row (oldest entry)
        row = data_list[-1]
        # Create fullname
        fullname = row['kind'] + '_' + row['id']
        # Add/update fullname in params
        params['after'] = fullname

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Insert data into the table
    for index, row in data.iterrows():
        cursor.execute('''INSERT INTO reddit_posts
                          (subreddit, title, selftext, upvote_ratio, ups, downs, score,
                           link_flair_css_class, created_utc, kind)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                       (row['subreddit'], row['title'], row['selftext'], row['upvote_ratio'],
                        row['ups'], row['downs'], row['score'], row['link_flair_css_class'],
                        row['created_utc'], row['kind']))

    # Commit changes
    conn.commit()

In [47]:
# Connect to SQLite database (create if it doesn't exist)
conn = sqlite3.connect('reddit_data.db')

In [48]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [49]:
# Create a table to store your data
cursor.execute('''CREATE TABLE IF NOT EXISTS reddit_posts (
                    id INTEGER PRIMARY KEY,
                    subreddit TEXT,
                    title TEXT,
                    selftext TEXT,
                    upvote_ratio REAL,
                    ups INTEGER,
                    downs INTEGER,
                    score INTEGER,
                    link_flair_css_class TEXT,
                    created_utc INTEGER,
                    kind TEXT,
                    cleaned TEXT
                )''')

In [50]:
def extract_and_add_to_database(subreddit_name, conn):
    # Initialize dataframe and parameters for pulling data in loop
    data = pd.DataFrame()
    params = {'limit': 100}

    # Loop through 10 times (returning 1K posts)
    for i in range(10):
        # Make request to the specified subreddit
        res = requests.get(f"https://oauth.reddit.com/r/{subreddit_name}/new",
                           headers=headers,
                           params=params)

        # Get list of dictionaries from response
        data_list = df_from_response(res)

        # Convert list of dictionaries to DataFrame and append to data
        data = pd.concat([data, pd.DataFrame(data_list)], ignore_index=True)

        # Take the final row (oldest entry)
        row = data_list[-1]
        # Create fullname
        fullname = row['kind'] + '_' + row['id']
        # Add/update fullname in params
        params['after'] = fullname

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Insert data into the table
    for index, row in data.iterrows():
        cursor.execute('''INSERT INTO reddit_posts
                          (subreddit, title, selftext, upvote_ratio, ups, downs, score,
                           link_flair_css_class, created_utc, kind)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                       (row['subreddit'], row['title'], row['selftext'], row['upvote_ratio'],
                        row['ups'], row['downs'], row['score'], row['link_flair_css_class'],
                        row['created_utc'], row['kind']))

    # Commit changes
    conn.commit()





In [51]:
# Connect to SQLite database
conn = sqlite3.connect('reddit_data.db')

# Extract data from a specific subreddit and add it to the database
extract_and_add_to_database('worldnews', conn)



- politics
- worldnews
- news
- energy
- environment
- business
- worldevents

In [53]:
extract_and_add_to_database('politics', conn)


In [55]:
extract_and_add_to_database('worldevents', conn)


In [56]:
extract_and_add_to_database('ukpolitics', conn)


In [59]:
extract_and_add_to_database('AskReddit', conn)


In [60]:
# Close connection
conn.close()